In [1]:
from aocd import get_data
from aocd import submit
import unittest

day = 3
year = 2023

def submit_part_a(answer):
    submit(answer, part="a", day=day, year=year)

def submit_part_b(answer):
    submit(answer, part="b", day=day, year=year)

input = get_data(day=day, year=year)

In [2]:
matrix = [[c for c in list(line)] for line in input.split('\n')]

In [3]:
# Just create the prod operation to be used like sum() but to multiply
from functools import reduce
import operator
def prod(iterable):
    return reduce(operator.mul, iterable, 1)

In [4]:
class Part:
    def __init__(self, number, symbol, symbol_x, symbol_y):
        self.number = number
        self.symbol = symbol
        self.symbol_x = symbol_x
        self.symbol_y = symbol_y

    def is_adjacent_to_gear(self):
        return self.symbol == '*'

    def get_symbol_coord(self):
        return self.symbol_x, self.symbol_y


class Schematic:
    digits_and_nothing = "0123456789."

    def __init__(self, matrix):
        self.matrix = matrix

    def get_part_numbers(self):
        return [part.number for part in self.get_parts()]

    def get_gear_ratios(self):
        parts_adjecent_to_a_gear = [part for part in self.get_parts() if part.is_adjacent_to_gear()]
        gear_ratios = []
        for y in range(len(self.matrix)):
            for x in range(len(self.matrix[y])):
                if self.matrix[y][x] == '*':
                    parts_adjacent_to_this_gear = [part.number for part in parts_adjecent_to_a_gear if part.get_symbol_coord() == (x, y)]
                    if len(parts_adjacent_to_this_gear) == 2:
                        gear_ratios.append(prod(parts_adjacent_to_this_gear))
        return gear_ratios
    
    def get_parts(self):
        parts = []
        for y in range(len(self.matrix)):
            current_part_number = 0
            symbol = None
            symbol_x = None
            symbol_y = None
            for x in range(len(self.matrix[y])):
                current_char = self.matrix[y][x]
                if current_char.isdigit():
                    current_part_number = current_part_number * 10 + int(current_char)
                    if symbol is None:
                        symbol, symbol_x, symbol_y = self.get_symbol_arround(x, y)
                else:
                    if symbol is not None:
                        parts.append(Part(current_part_number, symbol, symbol_x, symbol_y))
                    current_part_number = 0
                    symbol, symbol_x, symbol_y = None, None, None
            # Manage case when last char of line is a digit
            if symbol is not None:
                parts.append(Part(current_part_number, symbol, symbol_x, symbol_y))
            current_part_number = 0
            symbol, symbol_x, symbol_y = None, None, None
        return parts

    def get_symbol_arround(self, x, y):
        if self.has_symbol_at(x-1, y-1):
            return self.matrix[y-1][x-1], x-1, y-1
        if self.has_symbol_at(x, y-1):
            return self.matrix[y-1][x], x, y-1
        if self.has_symbol_at(x+1, y-1):
            return self.matrix[y-1][x+1], x+1, y-1
        if self.has_symbol_at(x-1, y):
            return self.matrix[y][x-1], x-1, y
        if self.has_symbol_at(x+1, y):
            return self.matrix[y][x+1], x+1, y
        if self.has_symbol_at(x-1, y+1):
            return self.matrix[y+1][x-1], x-1, y+1
        if self.has_symbol_at(x, y+1):
            return self.matrix[y+1][x], x, y+1, 
        if self.has_symbol_at(x+1, y+1):
            return self.matrix[y+1][x+1], x+1, y+1
        return None, None, None

    
    def has_symbol_arround(self, x, y):
        return (self.has_symbol_at(x-1, y-1)
                or self.has_symbol_at(x, y-1)
                or self.has_symbol_at(x+1, y-1)
                or self.has_symbol_at(x-1, y)
                or self.has_symbol_at(x+1, y)
                or self.has_symbol_at(x-1, y+1)
                or self.has_symbol_at(x, y+1)
                or self.has_symbol_at(x+1, y+1))

    def has_symbol_at(self, x, y):
        if (y < 0):
            return False
        if (y >= len(self.matrix)):
            return False
        if (x < 0):
            return False
        if (x >= len(self.matrix[0])):
            return False
        if self.matrix[y][x] not in self.digits_and_nothing:
            return True
        return False

In [5]:
class SchematicTest(unittest.TestCase):
    matrix = [
        ['4', '6', '7', '.', '.', '1', '1', '4', '.', '.'],
        ['.', '.', '.', '*', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '3', '5', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '#', '6', '3', '3'],
        ['6', '1', '7', '*', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '+', '.', '5', '8', '.'],
        ['.', '.', '5', '9', '2', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '7', '5', '5', '.'],
        ['.', '.', '.', '$', '.', '*', '.', '.', '.', '.'],
        ['.', '6', '6', '4', '.', '5', '9', '8', '.', '.']
    ]
    
    def test_0(self):
        schematic = Schematic(self.matrix)
        self.assertEqual(schematic.has_symbol_arround(0, 0), False)
        self.assertEqual(schematic.has_symbol_arround(1, 0), False)
        self.assertEqual(schematic.has_symbol_arround(2, 0), True)

    def test_1(self):
        schematic = Schematic(self.matrix)
        self.assertEqual(schematic.get_part_numbers(), [467, 35, 633, 617, 592, 755, 664, 598])

    def test_2(self):
        schematic = Schematic(self.matrix)
        self.assertEqual(sum(schematic.get_part_numbers()), 4361)


runner = unittest.TextTestRunner(verbosity=3)
res = runner.run(unittest.TestLoader().loadTestsFromTestCase(SchematicTest)) 
assert len(res.failures) == 0

test_0 (__main__.SchematicTest.test_0) ... ok
test_1 (__main__.SchematicTest.test_1) ... ok
test_2 (__main__.SchematicTest.test_2) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.012s

OK


In [6]:
%%time
schematic = Schematic(matrix)
first_answer = sum(schematic.get_part_numbers())
print(first_answer)

525181
CPU times: user 16.7 ms, sys: 3.3 ms, total: 20 ms
Wall time: 19.1 ms


In [7]:
submit_part_a(first_answer)

aocd will not submit that answer again. At 2023-12-03 12:34:59.100768-05:00 you've previously submitted 525181 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [8]:
class SchematicTest(unittest.TestCase):
    matrix = [
        ['4', '6', '7', '.', '.', '1', '1', '4', '.', '.'],
        ['.', '.', '.', '*', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '3', '5', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '#', '6', '3', '3'],
        ['6', '1', '7', '*', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '+', '.', '5', '8', '.'],
        ['.', '.', '5', '9', '2', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '7', '5', '5', '.'],
        ['.', '.', '.', '$', '.', '*', '.', '.', '.', '.'],
        ['.', '6', '6', '4', '.', '5', '9', '8', '.', '.']
    ]
    
    def test_1(self):
        schematic = Schematic(self.matrix)
        self.assertEqual(schematic.get_symbol_arround(2, 0), ('*', 3, 1))
        self.assertEqual(schematic.get_gear_ratios(), [16345, 451490])


runner = unittest.TextTestRunner(verbosity=3)
res = runner.run(unittest.TestLoader().loadTestsFromTestCase(SchematicTest)) 
assert len(res.failures) == 0

test_1 (__main__.SchematicTest.test_1) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


In [9]:
%%time
schematic = Schematic(matrix)
second_answer = sum(schematic.get_gear_ratios())
print(second_answer)

84289137
CPU times: user 85.4 ms, sys: 6.9 ms, total: 92.3 ms
Wall time: 89.2 ms


In [10]:
submit_part_b(second_answer)

aocd will not submit that answer again. At 2023-12-03 14:57:22.412632-05:00 you've previously submitted 84289137 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 3! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
